# Wildfires
## By: David Jean

Data can be found here:
https://www.kaggle.com/datasets/braddarrow/23-million-wildfires

In [37]:
#importing libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree, preprocessing
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, plot_roc_curve
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import os

import seaborn as sns

# plotly
# import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
#pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [38]:
# Set display to show up to 200 columns in dataframe

sns.reset_orig()
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows', 50)

# Set default color palette to colorblind
plt.style.use('seaborn-colorblind')

# Set the aesthetic style of any plots 
sns.set_style('whitegrid') 
sns.color_palette("colorblind")
%config InlineBackend.figure_format='retina'

# Enable the inline plotting
%matplotlib inline
# Trick to widen the screen
from IPython.core.display import display, HTML
#Widens the code landscape 
display(HTML("<style>.container { width:95% !important; }</style>"))

C:\Users\david\AppData\Local\Temp\ipykernel_7964\998000563.py:18: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



--------

In [39]:
os.getcwd()

'c:\\Users\\david\\Documents\\masters\\6330\\data'

--------------------

# 2nd Model: Predicting cost for the year 2021

# Supression Costs

In [40]:
#read in the supressions dataset
dfsup = pd.read_csv('Suppression.csv')

In [41]:
os.getcwd()

'c:\\Users\\david\\Documents\\masters\\6330\\data'

In [42]:
dfsup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           36 non-null     int64 
 1   Fires          36 non-null     object
 2   Acres          36 non-null     object
 3   ForestService  36 non-null     object
 4   DOIAgencies    36 non-null     object
 5   Total          36 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.8+ KB


In [43]:
dfsup

,Year,Fires,Acres,ForestService,DOIAgencies,Total
0,1985,"82,591","2,896,147","$161,505,000","$78,438,000","$239,943,000"
1,1986,"85,907","2,719,162","$111,625,000","$91,153,000","$202,778,000"
2,1987,"71,300","2,447,296","$253,657,000","$81,452,000","$335,109,000"
3,1988,"72,750","5,009,290","$429,609,000","$149,317,000","$578,926,000"
4,1989,"48,949","1,827,310","$331,672,000","$168,115,000","$499,787,000"
5,1990,"66,481","4,621,621","$253,700,000","$144,252,000","$397,952,000"
6,1991,"75,754","2,953,578","$132,300,000","$73,820,000","$206,120,000"
7,1992,"87,394","2,069,929","$290,300,000","$87,166,000","$377,466,000"
8,1993,"58,810","1,797,574","$184,000,000","$56,436,000","$240,436,000"
9,1994,"79,107","4,073,579","$757,200,000","$161,135,000","$918,335,000"


In [44]:
dfsup.describe()

,Year
count,36.000
mean,2002.500
std,10.536
min,1985.000
25%,1993.750
50%,2002.500
75%,2011.250
max,2020.000


In [45]:
dfsup['Fires'] = dfsup['Fires'].str.replace(',', '')
dfsup['Fires'] = dfsup['Fires'].astype(float)

In [46]:
#convert all the objects to floats
dfsup['Acres'] = dfsup['Acres'].str.replace(',', '')
dfsup['Acres'] = dfsup['Acres'].astype(float)

In [47]:
#convert all the objects to floats
dfsup['ForestService'] = dfsup['ForestService'].str.replace(',', '')
dfsup['ForestService'] = dfsup['ForestService'].str.replace('$', '')
dfsup['ForestService'] = dfsup['ForestService'].astype(float)

C:\Users\david\AppData\Local\Temp\ipykernel_7964\3732034004.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [48]:
dfsup['DOIAgencies'] = dfsup['DOIAgencies'].str.replace(',', '')
dfsup['DOIAgencies'] = dfsup['DOIAgencies'].str.replace('$', '')
dfsup['DOIAgencies'] = dfsup['DOIAgencies'].astype(float)

C:\Users\david\AppData\Local\Temp\ipykernel_7964\3726213583.py:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [49]:
dfsup['Total'] = dfsup['Total'].str.replace(',', '')
dfsup['Total'] = dfsup['Total'].str.replace('$', '')
dfsup['Total'] = dfsup['Total'].astype(float)

C:\Users\david\AppData\Local\Temp\ipykernel_7964\794900911.py:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [50]:
dfsup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           36 non-null     int64  
 1   Fires          36 non-null     float64
 2   Acres          36 non-null     float64
 3   ForestService  36 non-null     float64
 4   DOIAgencies    36 non-null     float64
 5   Total          36 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 1.8 KB


In [51]:
#enumerate a list of the columns
list(enumerate(dfsup.columns))

[(0, 'Year'),
 (1, 'Fires'),
 (2, 'Acres'),
 (3, 'ForestService'),
 (4, 'DOIAgencies'),
 (5, 'Total')]

In [52]:
X = dfsup.drop(['Total'], axis=1)
y = dfsup['Total']

In [53]:
#split the data into training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [54]:
model = XGBRegressor()

In [55]:
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [56]:
#show the X dataframe but with no e's
X_train.dtypes

Year               int64
Fires            float64
Acres            float64
ForestService    float64
DOIAgencies      float64
dtype: object

In [57]:
score = model.score(X_test, y_test)
score

0.9999999999999432

# Cerate a pipe to redo the model

In [58]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor())
])

In [59]:
#define the parameters for the grid search
params = {
    'xgb__learning_rate': [0.01, 0.1, 0.5],
    'xgb__max_depth': [3, 5, 7],
    'xgb__n_estimators': [100, 200, 500, 1000, 10000]
}

In [60]:
grid = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='neg_mean_squared_error')
results = grid.fit(X, y)

In [61]:
best_estimator = grid.best_estimator_

In [62]:
best_estimator

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=500,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [63]:
# Fit the model with the best estimator
best_estimator.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=500,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [64]:
#print df sup to show actual values and not science notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)
dfsup

,Year,Fires,Acres,ForestService,DOIAgencies,Total
0,1985,82591.000,2896147.000,161505000.000,78438000.000,239943000.000
1,1986,85907.000,2719162.000,111625000.000,91153000.000,202778000.000
2,1987,71300.000,2447296.000,253657000.000,81452000.000,335109000.000
3,1988,72750.000,5009290.000,429609000.000,149317000.000,578926000.000
4,1989,48949.000,1827310.000,331672000.000,168115000.000,499787000.000
5,1990,66481.000,4621621.000,253700000.000,144252000.000,397952000.000
6,1991,75754.000,2953578.000,132300000.000,73820000.000,206120000.000
7,1992,87394.000,2069929.000,290300000.000,87166000.000,377466000.000
8,1993,58810.000,1797574.000,184000000.000,56436000.000,240436000.000
9,1994,79107.000,4073579.000,757200000.000,161135000.000,918335000.000


In [65]:
dfsup.describe()

,Year,Fires,Acres,ForestService,DOIAgencies,Total
count,36.000,36.000,36.000,36.000,36.000,36.000
mean,2002.500,73118.694,5445069.722,865816638.889,267363138.889,1133179777.778
std,10.536,12723.249,2841679.836,636855540.690,149919541.418,777180717.260
min,1985.000,47579.000,1329704.000,111625000.000,56436000.000,202778000.000
25%,1993.750,66012.250,2939220.250,325454000.000,135720500.000,462020500.000
50%,2002.500,72360.500,4836827.000,714055500.000,275409000.000,936612500.000
75%,2011.250,82323.250,8245757.250,1216716250.000,396079750.000,1633618750.000
max,2020.000,96385.000,10125149.000,2615256000.000,528000000.000,3143256000.000


In [66]:
next_year_data = [[2021, 82323.250, 8245757.250, 1216716250.000, 396079750.000]]


In [67]:
# convert next_year_data to a dataframe with appropriate column names
next_year_df = pd.DataFrame(next_year_data, columns=['Year', 'Fires', 'Acres', 'ForestService', 'DOIAgencies'])

# make prediction using the trained model
next_year_prediction = best_estimator.predict(next_year_df)
print(next_year_prediction)

[1.5873699e+09]


In [68]:
#print next year prediction without it being in scientific notation and add commas in it

print(f'${next_year_prediction[0]:,.2f}')

$1,587,369,856.00


In [69]:
#put next year's predictions into a dataframe
next_year_df['Total'] = next_year_prediction

In [70]:
#concat it with dfsup
dfsup = pd.concat([dfsup, next_year_df], ignore_index=True)

In [71]:
dfsup

,Year,Fires,Acres,ForestService,DOIAgencies,Total
0,1985,82591.000,2896147.000,161505000.000,78438000.000,239943000.000
1,1986,85907.000,2719162.000,111625000.000,91153000.000,202778000.000
2,1987,71300.000,2447296.000,253657000.000,81452000.000,335109000.000
3,1988,72750.000,5009290.000,429609000.000,149317000.000,578926000.000
4,1989,48949.000,1827310.000,331672000.000,168115000.000,499787000.000
5,1990,66481.000,4621621.000,253700000.000,144252000.000,397952000.000
6,1991,75754.000,2953578.000,132300000.000,73820000.000,206120000.000
7,1992,87394.000,2069929.000,290300000.000,87166000.000,377466000.000
8,1993,58810.000,1797574.000,184000000.000,56436000.000,240436000.000
9,1994,79107.000,4073579.000,757200000.000,161135000.000,918335000.000
